In [ ]:
!git clone https://github.com/Dao-AILab/fast-hadamard-transform.git

In [ ]:
%cd fast-hadamard-transform
!pip install .

In [ ]:
from fast_hadamard_transform import hadamard_transform

In [ ]:
%cd /content

In [ ]:
!git clone https://github.com/Cornell-RelaxML/quip-sharp.git

In [ ]:

!pip install -r requirements.txt

In [ ]:
from transformers import pipeline
import torch
# Load the model pipeline
pipe = pipeline("text-generation", model="relaxml/Llama-2-7b-chat-QTIP-2Bit", trust_remote_code=True, torch_dtype=torch.float16, device_map="auto")

# Use a properly formatted string as input, with do_sample
response = pipe("Who are you?", max_new_tokens=22, do_sample=True)

# Print response
print(response[0]['generated_text'])

In [ ]:
from transformers import pipeline
import torch
# Load the model pipeline
pipe = pipeline("text-generation", model="relaxml/Llama-2-7b-chat-QTIP-2Bit", trust_remote_code=True, torch_dtype="auto", device_map="auto")

# Use a properly formatted string as input, with do_sample
response = pipe("Who are you?", max_new_tokens=22, do_sample=True)

# Print response
print(response[0]['generated_text'])

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
tokenizer = AutoTokenizer.from_pretrained("relaxml/Llama-2-7b-chat-QTIP-2Bit")
model = AutoModelForCausalLM.from_pretrained("relaxml/Llama-2-7b-chat-QTIP-2Bit", trust_remote_code=True, torch_dtype=torch.float16, device_map="auto")

# we use alpaca prompt
input_content = "hi"
input_ids = tokenizer.encode(input_content, return_tensors="pt")
output = model.generate(input_ids, max_length=11, temperature=0.7)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(output_text)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Explicitly download the tokenizer files
tokenizer = AutoTokenizer.from_pretrained("relaxml/Llama-2-7b-chat-QTIP-2Bit")

# Load the model
model = AutoModelForCausalLM.from_pretrained("relaxml/Llama-2-7b-chat-QTIP-2Bit", trust_remote_code=True, torch_dtype=torch.float16, device_map="auto")

# The rest of your code
input_content = "hi"
input_ids = tokenizer.encode(input_content, return_tensors="pt")
output = model.generate(input_ids, max_length=11, temperature=0.7)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(output_text)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# تحديد اسم النموذج
model_name = "relaxml/Llama-2-7b-chat-QTIP-2Bit"

# تحميل tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# تحميل النموذج
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

In [ ]:
from lib.utils.unsafe_import import model_from_hf_path
import torch

def get_quantized_layer(path2qmodel="YOURPATH"):
    # load a quantized model
    quant_model = model_from_hf_path(path2qmodel)[0].float()
    # select an arbitrary layer.
    quantized_layer = quant_model.model.layers[0].self_attn.q_proj

    # replicate the routine in finetune_e2e_llama.py #L95:107 with --ft_train_lut flag
    quantized_layer.SU = torch.nn.Parameter(quantized_layer.SU.float(), requires_grad=True)
    quantized_layer.SV = torch.nn.Parameter(quantized_layer.SV.float(), requires_grad=True)
    quantized_layer.mode = "train-recons"
    quantized_layer.tlut.requires_grad = True

    return quantized_layer

def test_backward():
    # load quantized layer
    quantized_layer = get_quantized_layer()

    # initialize random input to the layer
    ft_bs, ctx_size, in_features = 4, 4096, 4096
    input = torch.randn(ft_bs, ctx_size, in_features).to('cuda').to(torch.float16)
    input.requires_grad = True

    print("=== Before backward ===")
    print("input", input.grad)
    print("SU", quantized_layer.SU.grad)
    print("SV", quantized_layer.SV.grad)
    print("tlut", quantized_layer.tlut.grad)

    # forward pass
    output = quantized_layer(input)

    # backward pass
    loss = output.sum()
    loss.backward()

    print("=== After backward ===")
    print("input", input.grad)
    print("SU", quantized_layer.SU.grad)
    print("SV", quantized_layer.SV.grad)
    print("tlut", quantized_layer.tlut.grad)

if __name__ == "__main__":
    test_backward()

In [ ]:
 torch_dtype=torch.float16, device_map="auto"

In [ ]:
from transformers import pipeline
import torch
# Load the model pipeline
pipe = pipeline("text-generation", model="relaxml/Llama-2-7b-chat-QTIP-2Bit", trust_remote_code=True)

# Use a properly formatted string as input, with do_sample
response = pipe("Who are you?", max_new_tokens=22, do_sample=True)

# Print response
print(response[0]['generated_text'])

In [ ]:
import torch
assert torch.cuda.is_available(), "CUDA غير متاح، تأكد من أنك تستخدم T4!"


In [ ]:
import torch
print("CUDA متاح؟", torch.cuda.is_available())
print("عدد كروت الشاشة المتاحة:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("اسم كرت الشاشة:", torch.cuda.get_device_name(0))
    print("إصدار CUDA:", torch.version.cuda)
else:
    print("⚠️ CUDA غير متاح، تأكد من تمكين GPU في كولاب!")


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "relaxml/Llama-2-7b-chat-QTIP-2Bit"

# تحميل التوكنيزر
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

# تحميل النموذج على CUDA
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True).to("cuda")

# إدخال النص وتحويله إلى tensors
inputs = tokenizer("Who are you?", return_tensors="pt").to("cuda")

# توليد النص
output = model.generate(**inputs, max_new_tokens=22, do_sample=True)

# طباعة النتيجة
print(tokenizer.decode(output[0], skip_special_tokens=True))


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "mobiuslabsgmbh/Llama-3.2-3B-Instruct_4bitgs64_hqq_hf"

# تحميل التوكنيزر
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

# تحميل النموذج على CUDA
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True).to("cuda")

# إدخال النص وتحويله إلى tensors
inputs = tokenizer("Who are you?", return_tensors="pt").to("cuda")

# توليد النص
output = model.generate(**inputs, max_new_tokens=22, do_sample=True)

# طباعة النتيجة
print(tokenizer.decode(output[0], skip_special_tokens=True))


In [ ]:
!python /content/interactive_gen.py

In [ ]:
!huggingface-cli model info relaxml/Llama-2-7b-chat-QTIP-2Bit


In [ ]:
from transformers import pipeline
import torch
# Load the model pipeline
pipe = pipeline("text-generation", model="relaxml/Llama-1-7b-E8P-2Bit", trust_remote_code=True, torch_dtype=torch.float16, device_map="auto")

# Use a properly formatted string as input, with do_sample
response = pipe("Who are you?", max_new_tokens=22, do_sample=True)

# Print response
print(response[0]['generated_text'])

In [ ]:
import torch
from transformers import AutoTokenizer
from lib.utils.unsafe_import import model_from_hf_path  # التأكد من توفر `lib.utils.unsafe_import`

# تحقق من توافر CUDA
assert torch.cuda.is_available(), "CUDA غير متاح، تأكد من تفعيل GPU في كولاب!"

# تحميل النموذج من Hugging Face
model_id = "relaxml/Llama-2-7b-chat-QTIP-2Bit"
model, model_str = model_from_hf_path(model_id, use_cuda_graph=False, device_map="cuda:0")

# تحميل التوكنيزر
tokenizer = AutoTokenizer.from_pretrained(model_str)
tokenizer.pad_token = tokenizer.eos_token

# إعداد التخزين المؤقت للنموذج
from transformers import StaticCache
model._setup_cache(StaticCache, 1, max_cache_len=2048)

@torch.no_grad()
def generate_text(prompt, max_new_tokens=100, top_k=5, temperature=0.6):
    """ استدلال النص باستخدام النموذج """
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    cache_position = torch.arange(inputs["input_ids"].shape[1], device=model.device)

    logits = model(**inputs, cache_position=cache_position, return_dict=False, use_cache=True)[0]
    probs = torch.nn.functional.softmax(logits[:, -1] / temperature, dim=-1)
    next_token = torch.multinomial(probs, num_samples=1)

    generated_tokens = [next_token.item()]
    for _ in range(max_new_tokens - 1):
        cache_position += 1
        logits = model(next_token, cache_position=cache_position, return_dict=False, use_cache=True)[0]
        probs = torch.nn.functional.softmax(logits[:, -1] / temperature, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)
        generated_tokens.append(next_token.item())

    return tokenizer.decode(generated_tokens, skip_special_tokens=True)

# تجربة النموذج
prompt = "Who are you?"
output = generate_text(prompt, max_new_tokens=50, top_k=5, temperature=0.7)

print("📝 النموذج يقول:", output)


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# تحقق من توافر CUDA
assert torch.cuda.is_available(), "CUDA غير متاح، تأكد من تفعيل GPU في كولاب!"

# تحميل النموذج والتوكنيزر
model_id = "relaxml/Llama-2-7b-chat-QTIP-2Bit"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True).to("cuda")

@torch.no_grad()
def generate_text(prompt, max_new_tokens=100, top_k=5, temperature=0.6):
    """ استدلال النص باستخدام النموذج """
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    output_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        top_k=top_k,
        temperature=temperature,
        do_sample=True
    )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# تجربة النموذج
prompt = "Who are you?"
output = generate_text(prompt, max_new_tokens=50, top_k=5, temperature=0.7)

print("📝 النموذج يقول:", output)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "relaxml/Llama-2-7b-chat-QTIP-2Bit"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True).to("cpu")

prompt = "Who are you?"
inputs = tokenizer(prompt, return_tensors="pt").to("cpu")

output_ids = model.generate(**inputs, max_new_tokens=50, do_sample=True)
output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("📝 النموذج يقول:", output)


In [ ]:
import os

model_path = "/root/.cache/huggingface/hub/models--relaxml--Llama-2-7b-chat-QTIP-2Bit/snapshots/1725daafaf5c630feedeb27c9a277eba9874f417"

if os.path.exists(model_path):
    print("✅ النموذج موجود في المسار:", model_path)
    print("📂 الملفات الموجودة في المجلد:")
    print(os.listdir(model_path))
else:
    print("❌ النموذج غير موجود، تأكد من تحميله بشكل صحيح!")


In [ ]:
from huggingface_hub import hf_hub_download

model_id = "relaxml/Llama-2-7b-chat-QTIP-2Bit"
local_model_path = "/content/relaxml_model"

# تحميل ملفات التوكنيزر الناقصة
hf_hub_download(repo_id=model_id, filename="tokenizer.json", local_dir=local_model_path)
hf_hub_download(repo_id=model_id, filename="tokenizer_config.json", local_dir=local_model_path)
hf_hub_download(repo_id=model_id, filename="special_tokens_map.json", local_dir=local_model_path)


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(local_model_path, trust_remote_code=True)
print("✅ تم تحميل التوكنيزر بنجاح!")


In [ ]:
from huggingface_hub import snapshot_download

model_id = "relaxml/Llama-2-7b-chat-E8PRVQ-4Bit"
local_model_path = "/content/relaxml_model"

# تحميل جميع ملفات النموذج إلى المجلد
snapshot_download(repo_id=model_id, local_dir=local_model_path)

print(f"✅ تم تحميل النموذج بالكامل إلى: {local_model_path}")


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

local_model_path = "/content/relaxml_model"
#model = AutoModelForCausalLM.from_pretrained(local_model_path, trust_remote_code=True).to("cpu")

# تحميل التوكنيزر والنموذج من المجلد
tokenizer = AutoTokenizer.from_pretrained(local_model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(local_model_path, trust_remote_code=True).to("cpu")

@torch.no_grad()
def generate_text(prompt, max_new_tokens=1, top_k=5, temperature=0.6):
    """ استدلال النص باستخدام النموذج """
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    output_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        top_k=top_k,
        temperature=temperature,
        do_sample=True
    )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# تجربة النموذج
prompt = "Who are you?"
output = generate_text(prompt, max_new_tokens=50, top_k=5, temperature=0.7)

print("📝 النموذج يقول:", output)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

local_model_path = "/content/relaxml_model"

# تحميل التوكنيزر
tokenizer = AutoTokenizer.from_pretrained(local_model_path, trust_remote_code=True)

# تحميل النموذج مع توزيع الأجزاء تلقائيًا بين CPU و GPU
model = AutoModelForCausalLM.from_pretrained(
    local_model_path,
    trust_remote_code=True,
    device_map="auto"  # يجعل بعض الأجزاء تعمل على CPU لتوفير VRAM
)

print("✅ تم تحميل النموذج بنجاح!")
